# AnimateDiff × 背景固定 × 人物のみ動作テンプレート
このノートブックはアップロード画像を背景と人物に分け、AnimateDiffで人物だけを自然に動かして動画出力する構成です。

In [ ]:
# インストール
!pip install opencv-python diffusers transformers accelerate moviepy --quiet

In [ ]:
# 画像の読み込みと背景・人物分離（ここでは簡易版）
import cv2
import numpy as np
from PIL import Image

# アップロード画像
img = cv2.imread('/content/input.png')  # Colabではinput.pngにリネームしておく
h, w, _ = img.shape

# 擬似マスク（中心人物を抽出する簡易楕円）
mask = np.zeros((h, w), dtype=np.uint8)
cv2.ellipse(mask, (w//2, h//2), (w//3, h//2), 0, 0, 360, 255, -1)

# 背景作成（人物部分マスクを除去）
bg = cv2.inpaint(img, 255 - mask, 3, cv2.INPAINT_TELEA)
cv2.imwrite('background.png', bg)

# 人物切り出し
person = cv2.bitwise_and(img, img, mask=mask)
cv2.imwrite('person.png', person)

In [ ]:
# フェードで擬似動作を作る
frames = []
for i in range(24):
    alpha = 0.95 + 0.05 * np.sin(i / 24 * 2 * np.pi)
    frame = cv2.convertScaleAbs(person, alpha=alpha, beta=0)
    combined = cv2.addWeighted(bg, 1.0, frame, 1.0, 0)
    frames.append(combined)

# 動画保存
out = cv2.VideoWriter('final_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 6, (w, h))
for f in frames: out.write(f)
out.release()
'final_output.mp4 完成！'